In [11]:
import pandas as pd

In [12]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoConfig
from tsfm_public.models.tspulse.modeling_tspulse import TSPulseForClassification
from collections import OrderedDict
from safetensors.torch import load_file
import os
import logging
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Configure logging to show all info
logging.basicConfig(level=logging.INFO, format='INFO:%(name)s:%(message)s')
logger = logging.getLogger(__name__)


In [13]:

# --- Functions ---
def load_tspulse_model(ckpt: str, model_class, config=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if config is None:
        config = AutoConfig.from_pretrained(ckpt, trust_remote_code=True)
    
    st_path = os.path.join(ckpt, "model.safetensors")
    bin_path = os.path.join(ckpt, "pytorch_model.bin")
    
    state_dict = None
    if os.path.exists(st_path):
        state_dict = load_file(st_path, device="cpu")
    elif os.path.exists(bin_path):
        state_dict = torch.load(bin_path, map_location="cpu")
    
    if state_dict is not None:
        new_sd = OrderedDict()
        for k, v in state_dict.items():
            nk = k.replace("_orig_mod.", "", 1) if k.startswith("_orig_mod.") else k
            new_sd[nk] = v
        model = model_class(config).to(device)
        model.load_state_dict(new_sd, strict=False)
    else:
        raise FileNotFoundError(f"No model file found in checkpoint directory: {ckpt}")
    
    return model

In [14]:
class ClassificationDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return { "past_values": self.X[idx], "target_values": self.y[idx] }

def evaluate_model(model, dataloader, device, X_test_raw, label_name, num_labels):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'target_values'}
            labels = batch['target_values'].to(device)
            
            inputs['past_observed_mask'] = (inputs['past_values'] != 0.0)
            outputs = model(**inputs) 

            logits = outputs.prediction_outputs
            if logits.dim() > 2:
                logits = logits.mean(dim=1)
            
            if labels.dim() > 1:
                labels = labels[:, 0]
            labels = labels.view(-1)
            predictions = torch.argmax(logits, dim=-1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    if np.any(all_labels == 1):
        true_mean_label_1 = X_test_raw[all_labels == 1].mean()
    else:
        true_mean_label_1 = np.nan

    if np.any(all_preds == 1):
        predicted_mean_label_1 = X_test_raw[all_preds == 1].mean()
    else:
        predicted_mean_label_1 = np.nan

    predicted_positives_count = np.sum(all_preds == 1)

    print("\n" + "="*40)
    print(f"Classification Report for: {label_name}")
    print("="*40)
    all_possible_labels = list(range(num_labels))
    target_names = [f"class_{i}" for i in all_possible_labels]
    report_dict = classification_report(
    all_labels,
    all_preds,
    labels=all_possible_labels,
    target_names=target_names,
    zero_division=0,
    output_dict=True
)

    report_text = classification_report(
        all_labels,
        all_preds,
        labels=all_possible_labels,
        target_names=target_names,
        zero_division=0
    )
    print(report_text)
    print("="*40 + "\n")

    # Extract macro precision and recall
    precision_macro = report_dict["macro avg"]["precision"]
    recall_macro = report_dict["macro avg"]["recall"]

    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "true_mean_label_1": true_mean_label_1,
        "predicted_mean_label_1": predicted_mean_label_1,
        "predicted_positives_count": predicted_positives_count
    }


In [ ]:


# --- Data Loading and Preprocessing ---
df = pd.read_csv('./Dataset/Classification/comstock_60min_small.csv')
labels_df = pd.read_csv('./Dataset/Classification/comstock_60min_labels.csv')

X_raw = df.drop(columns=[ "Timestamp"]).T.values  # "Unnamed: 0",
context_length = 512
num_series, series_length = X_raw.shape
X_processed = np.zeros((num_series, context_length))

for i in tqdm(range(num_series), desc="Processing Series"):
    series = X_raw[i, :]
    series = np.nan_to_num(series, nan=0.0)
    if len(series) > context_length:
        X_processed[i, :] = series[:context_length]
    else:
        X_processed[i, :len(series)] = series
X_processed = X_processed[:, :, np.newaxis]

# --- Main Training Loop ---
target_labels = ["cooling_ON", "fans_ON", "heat_rejection_ON", "heating_ON", "refrigeration_ON", "water_systems_ON"]
pretrained_model_path = "./Model-Checkpoints/final_model"
base_output_dir = "./finetuned_classification_manual_loop"
os.makedirs(base_output_dir, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
all_results = {}


Processing Series:   0%|          | 0/1000 [00:00<?, ?it/s]

In [21]:
config = AutoConfig.from_pretrained(pretrained_model_path, trust_remote_code=True)


In [22]:
config

TSPulseConfig {
  "architectures": [
    "TSPulseForReconstruction"
  ],
  "batch_aware_masking": false,
  "categorical_vocab_size_list": null,
  "channel_consistent_masking": true,
  "channel_mix_init": "identity",
  "channel_register_tokens": null,
  "channel_virtual_expand_scale": 1,
  "classification_mode": "full_embedding",
  "context_length": 512,
  "d_model": 16,
  "d_model_layerwise": [
    16,
    16,
    16
  ],
  "d_model_layerwise_scale": [
    1,
    1,
    1
  ],
  "data_actual_context_length": null,
  "decoder_d_model": 16,
  "decoder_d_model_layerwise": [
    16,
    16,
    16
  ],
  "decoder_d_model_layerwise_scale": [
    1,
    1,
    1
  ],
  "decoder_mode": "common_channel",
  "decoder_num_channels_layerwise": [
    1,
    1,
    1
  ],
  "decoder_num_channels_layerwise_scale": [
    1,
    1,
    1
  ],
  "decoder_num_layers": 3,
  "decoder_num_patches_layerwise": [
    64,
    64,
    64
  ],
  "decoder_num_patches_layerwise_scale": [
    1,
    1,
    1
  ],
  

In [23]:
for label_name in target_labels:
    logger.info("="*80)
    logger.info(f"Starting fine-tuning for label: {label_name}")
    
    if label_name not in labels_df.columns:
        logger.warning(f"Label '{label_name}' not found in labels file. Skipping.")
        continue
        
    y_current = labels_df[label_name].values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_current)
    num_labels = len(np.unique(y_encoded))
    
    logger.info(f"Label '{label_name}' has {num_labels} unique classes: {np.unique(y_encoded)}")
    if num_labels <= 1:
        logger.warning(f"Skipping label '{label_name}' as it has only one class.")
        continue

    X_train, X_val, y_train, y_val = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    
    train_dataset = ClassificationDataset(X_train, y_train)
    val_dataset = ClassificationDataset(X_val, y_val)
    X_test_raw_for_eval = X_val.squeeze()
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # Manually calculate class weights
    y_train_flat = y_train.ravel()
    present_classes = np.unique(y_train_flat)
    final_weights = torch.ones(num_labels, dtype=torch.float, device=device)
    if len(present_classes) > 1:
        class_counts = np.bincount(y_train_flat)
        n_train_samples = len(y_train_flat)
        num_present_classes = len(present_classes)
        for class_idx in present_classes:
            if class_idx < len(class_counts):
                weight = n_train_samples / (num_present_classes * class_counts[class_idx] + 1e-6)
                final_weights[class_idx] = weight
    
    logger.info(f"Final weight tensor for all {num_labels} classes: {final_weights.cpu().numpy()}")
    loss_fct = torch.nn.CrossEntropyLoss() #weight=final_weights

    config = AutoConfig.from_pretrained(pretrained_model_path, trust_remote_code=True)
    config.num_labels = num_labels
     # pretrained_model_path,
        # config=config,
    model = TSPulseForClassification.from_pretrained(
        pretrained_model_path,
        config=config,
        ignore_mismatched_sizes=True,
    )
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    
    num_epochs = 200 # You can increase this
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'target_values'}
            labels = batch['target_values'].to(device).view(-1)
            optimizer.zero_grad()
            
            inputs['past_observed_mask'] = (inputs['past_values'] != 0.0)
            outputs = model(**inputs)
            
            logits = outputs.prediction_outputs
            if logits.dim() > 2:
                logits = logits.mean(dim=1)
            
            loss = loss_fct(logits, labels)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
        
        avg_loss = epoch_loss / len(train_dataloader)
        eval_results = evaluate_model(model, val_dataloader, device, X_test_raw_for_eval, label_name, num_labels)
        logger.info(f"Epoch {epoch+1} - Avg. Training Loss: {avg_loss:.4f} - Validation Accuracy: {eval_results['accuracy']:.4f} - Validation Macro F1: {eval_results['f1_macro']:.4f}")

    logger.info(f"Final evaluation for label: {label_name}")
    final_test_results = evaluate_model(model, val_dataloader, device, X_test_raw_for_eval, label_name, num_labels)
    all_results[label_name] = final_test_results
    
    final_model_path = os.path.join(base_output_dir, f"model_{label_name}")
    model.save_pretrained(final_model_path)
    logger.info(f"Model for '{label_name}' saved to: {final_model_path}\n\n")

# --- Final Summary ---
logger.info("="*80)
logger.info("Final Classification Summary for All Labels")
logger.info("="*80)
results_df = pd.DataFrame(all_results).T
print(results_df)

summary_filename = "final_classification_summary_manual_BuildSys_15min.txt"
summary_string = results_df.to_string()
try:
    with open(summary_filename, "w") as f:
        f.write("Final Classification Summary (Manual Loop)\n")
        f.write("==========================================\n\n")
        f.write(summary_string)
    logger.info(f"Final summary successfully saved to: {summary_filename}")
except Exception as e:
    logger.error(f"Failed to save summary file. Error: {e}")

INFO:p-7653:t-134856693540672:2101627701.py:<module>:================================================================================
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Starting fine-tuning for label: cooling_ON
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Label 'cooling_ON' has 2 unique classes: [0 1]
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Final weight tensor for all 2 classes: [2.1164021  0.65466446]
INFO:p-7653:t-134856693540672:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-7653:t-134856693540672:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-7653:t-134856693540672:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-7653:t-134856693540672:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-7653:t-134856693540672:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: py

Epoch 1/200:   0%|          | 0/25 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:p-7653:t-134856693540672:2101627701.py:<module>:Epoch 1 - Avg. Training Loss: 1.0978 - Validation Accuracy: 0.4700 - Validation Macro F1: 0.2338



Classification Report for: cooling_ON
              precision    recall  f1-score   support

     class_0       0.00      0.00      0.00        47
     class_1       0.82      0.61      0.70       153

   micro avg       0.80      0.47      0.59       200
   macro avg       0.41      0.31      0.35       200
weighted avg       0.63      0.47      0.54       200




...

INFO:p-7653:t-134856693540672:2101627701.py:<module>:Epoch 200 - Avg. Training Loss: 0.5332 - Validation Accuracy: 0.7350 - Validation Macro F1: 0.4578
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Final evaluation for label: water_systems_ON



Classification Report for: water_systems_ON
              precision    recall  f1-score   support

     class_0       1.00      0.04      0.07        55
     class_1       0.73      1.00      0.85       145

    accuracy                           0.73       200
   macro avg       0.87      0.52      0.46       200
weighted avg       0.81      0.73      0.63       200




Evaluating:   0%|          | 0/7 [00:00<?, ?it/s]

INFO:p-7653:t-134856693540672:2101627701.py:<module>:Model for 'water_systems_ON' saved to: ./finetuned_classification_manual_loop/model_water_systems_ON


INFO:p-7653:t-134856693540672:2101627701.py:<module>:================================================================================
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Final Classification Summary for All Labels
INFO:p-7653:t-134856693540672:2101627701.py:<module>:================================================================================
INFO:p-7653:t-134856693540672:2101627701.py:<module>:Final summary successfully saved to: final_classification_summary_manual_BuildSys_15min.txt



Classification Report for: water_systems_ON
              precision    recall  f1-score   support

     class_0       1.00      0.04      0.07        55
     class_1       0.73      1.00      0.85       145

    accuracy                           0.73       200
   macro avg       0.87      0.52      0.46       200
weighted avg       0.81      0.73      0.63       200


                   accuracy  f1_macro  precision_macro  recall_macro  \
cooling_ON            0.830  0.723847         0.786046      0.697260   
fans_ON               0.900  0.694750         0.717391      0.677778   
heat_rejection_ON     0.885  0.469496         0.442500      0.500000   
heating_ON            0.780  0.769850         0.794118      0.766667   
refrigeration_ON      0.770  0.720738         0.734951      0.712084   
water_systems_ON      0.735  0.457828         0.866162      0.518182   

                   true_mean_label_1  predicted_mean_label_1  \
cooling_ON                  7.146098                7.0673

Processing Series:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:p-1487422:t-131857727276864:292093886.py:<module>:================================================================================
INFO:p-1487422:t-131857727276864:292093886.py:<module>:Starting fine-tuning for label: cooling_ON
INFO:p-1487422:t-131857727276864:292093886.py:<module>:Label 'cooling_ON' has 2 unique classes: [0 1]
INFO:p-1487422:t-131857727276864:292093886.py:<module>:Final weight tensor for all 2 classes: [2.1164021  0.65466446]
INFO:p-1487422:t-131857727276864:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-1487422:t-131857727276864:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-1487422:t-131857727276864:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-1487422:t-131857727276864:modeling_tspulse.py:_init_weights:Initializing Linear layers with method: pytorch
INFO:p-1487422:t-131857727276864:modeling_tspulse.py:_init_weights:Initializing Linear

Epoch 1/20:   0%|          | 0/25 [00:00<?, ?it/s]

RuntimeError: The size of tensor a (64) must match the size of tensor b (72) at non-singleton dimension 2